# Práctica 03 — Información Mutua

Este notebook calcula las asociaciones de palabras (PMI) por dataset tomando la columna `title`.

## 0) Imports y configuración

In [2]:
from pathlib import Path
import sys
import pandas as pd

PROJ = Path.cwd().parent           
SCRIPTS_DIR = Path.cwd()           
DATA = PROJ / "data"               

from scripts.text_preprocess import preprocess_text, tokenize_words
from scripts.ngrams import ngrams as ngrams_fn  
from scripts.io_titles import read_titles_csv, join_titles
from scripts.pmi import pmi_df, bigram_counter_from_str_keys
from scripts.report import top_pmi, save_table

# Archivos
df_proceso_titles = pd.read_csv(DATA / "noticias_proceso.csv")
df_elpais_titles = pd.read_csv(DATA / "noticias_elpais.csv")
df_elfinanciero_titles = pd.read_csv(DATA / "noticias_elfinanciero.csv")

MIN_FREQ = 10
TOP_K = 10


## Se va a separar por archivo cada preprocesamiento y calculo de PMI

### 1.1) PROCESO

#### Preprocesamiento de texto

In [3]:
df_proceso_titles["title_clean"] = df_proceso_titles["title"].apply(preprocess_text)

In [4]:
df_proceso_titles.head(3)

,title,year,month,day,pubDate,link,description,image_url,image_caption,cluster,Emoción,title_clean
0,Pretenden juzgar dos veces por el mismo delito...,2023,3,28,2023-03-28T11:46:01-00:00,https://www.proceso.com.mx/nacional/2023/3/28/...,"El activista fue encarcelado en enero de 2020,...",https://www.proceso.com.mx/u/fotografias/m/202...,"El activista fue encarcelado en enero de 2020,...",2,🤔,pretenden juzgar dos veces por el mismo delito...
1,AMLO anuncia campaña contra las drogas en escu...,2023,3,28,2023-03-28T11:25:02-00:00,https://www.proceso.com.mx/nacional/2023/3/28/...,"“Si te drogas te dañas”, es el nombre de la ca...",https://www.proceso.com.mx/u/fotografias/m/202...,"“Si te drogas te dañas”, es el nombre de la ca...",3,🤔,amlo anuncia campana contra las drogas en escu...
2,Informe &quot;Voces contra la indiferencia&quot;,2023,3,28,2023-03-28T11:15:03-00:00,https://www.proceso.com.mx/nacional/2023/3/28/...,NaN,https://www.proceso.com.mx/u/fotografias/m/202...,NaN,4,❤,informe &quot voces contra la indiferencia&quot


#### Medida de información mutua

In [5]:
text_proceso = " ".join(df_proceso_titles["title_clean"].tolist())

# Tokenizamos
tokens_proceso = tokenize_words(text_proceso)
print(f"Número de tokens: {len(tokens_proceso)}")

bigrams_str_proceso = ngrams_fn(tokens_proceso, n=2)

# Obtenemos los bigramas
bigrams_proceso = bigram_counter_from_str_keys(bigrams_str_proceso)
print(f"Número de bigramas: {len(bigrams_proceso)}")

# Calculamos el PMI
pmi_proceso = pmi_df(tokens_proceso, bigrams_proceso, min_freq=MIN_FREQ)

# Mostramos los top-k
top_pmi(pmi_proceso, k=TOP_K)

Número de tokens: 55444
Número de bigramas: 31449


,w1,w2,freq,pmi
0,t,mec,11,12.299338
1,salinas,pliego,10,11.795296
2,elena,poniatowska,11,11.710407
3,delfina,gomez,11,11.710407
4,conferencia,mananera,11,11.503956
5,tren,maya,13,11.459209
6,adan,augusto,19,11.436842
7,cambio,climatico,10,11.235208
8,moreno,valle,11,11.141386
9,movimiento,ciudadano,12,11.086344


### 1.2) ELPAIS

#### Preprocesamiento de texto

In [6]:
df_elpais_titles["title_clean"] = df_elpais_titles["title"].apply(preprocess_text)

In [7]:
df_elfinanciero_titles.head(3)

,title,year,month,day,pubDate,link,description,image_url,image_caption,cluster,Emoción
0,AMLO: otra duda sobre su cardenismo,2023,3,28,"Tue, 28 Mar 2023 18:48:49 +0000",https://www.elfinanciero.com.mx/opinion/salvad...,Es común que Lázaro Cárdenas sea utilizado par...,https://elfinanciero-elfinanciero-prod.cdn.arc...,Andrés Manuel López Obrador quiere usar al gen...,1,🤔
1,Esto sabemos de la muerte de 39 migrantes en u...,2023,3,28,"Tue, 28 Mar 2023 18:42:18 +0000",https://www.elfinanciero.com.mx/estados/2023/0...,La FGR confirmó la muerte de 39 migrantes tras...,https://elfinanciero-elfinanciero-prod.cdn.arc...,"Antes del incidente, los agentes del INM había...",4,🤔
2,Ricardo Monreal instala primer comité de proye...,2023,3,28,"Tue, 28 Mar 2023 18:23:15 +0000",https://www.elfinanciero.com.mx/nacional/2023/...,El senador ha defendido dejar de lado la confr...,https://elfinanciero-elfinanciero-prod.cdn.arc...,Monreal es una de las llamadas 'corcholatas' d...,1,❤


#### Medida de información mutua

In [8]:
text_pais = " ".join(df_elpais_titles["title_clean"].tolist())

tokens_pais = tokenize_words(text_pais)
print(f"Número de tokens: {len(tokens_pais)}")

bigrams_str_pais = ngrams_fn(tokens_pais, n=2)

bigrams_pais = bigram_counter_from_str_keys(bigrams_str_pais)
print(f"Número de bigramas: {len(bigrams_pais)}")

pmi_pais = pmi_df(tokens_pais, bigrams_pais, min_freq=MIN_FREQ)

# Mostramos los top-k
top_pmi(pmi_pais, k=TOP_K)

Número de tokens: 443328
Número de bigramas: 160560


,w1,w2,freq,pmi
0,dorados,llamativos,10,15.436090
1,svenja,flasspohler,10,15.436090
2,‘te,deseo”,10,15.436090
3,nagorno,karabaj,11,15.298587
4,shikoku,henro,11,15.298587
5,hispanics,shaping,11,15.298587
6,brad,pitt,10,15.298587
7,skid,row,10,15.298587
8,jada,pinkett,10,15.298587
9,ethan,hawke,12,15.173056


### ELFINANCIERO

#### Preprocesamiento

In [9]:

df_elfinanciero_titles["title_clean"] = df_elfinanciero_titles["title"].apply(preprocess_text)

In [10]:
df_elpais_titles.head(3)

,title,year,month,day,pubDate,link,description,image_url,image_caption,cluster,Emoción,title_clean
0,"Javier Milei, el político argentino inclasific...",2023,3.0,28.0,"Tue, 28 Mar 2023 10:15:00 GMT",https://elpais.com/argentina/2023-03-28/javier...,El economista que encarna el trumpismo sudamer...,https://cloudfront-eu-central-1.images.arcpubl...,"El diputado Javier Milei, líder de La Libertad...",0,🤔,javier milei el politico argentino inclasificable
1,Maduro volverá a la mesa de México si se liber...,2023,3.0,28.0,"Tue, 28 Mar 2023 10:15:00 GMT",https://elpais.com/internacional/2023-03-28/ma...,"Estados Unidos, por su parte, insiste en que s...",https://cloudfront-eu-central-1.images.arcpubl...,"Nicolás Maduro, presidente de Venezuela, duran...",1,🤔,maduro volvera a la mesa de mexico si se liber...
2,La recuperación en Venezuela es solo para unos...,2023,3.0,28.0,"Tue, 28 Mar 2023 03:45:00 GMT",https://elpais.com/economia/negocios/2023-03-2...,Los ingresos petrolíferos y la liberalización ...,https://cloudfront-eu-central-1.images.arcpubl...,Profesores marchan en una protesta contra la c...,4,❤,la recuperacion en venezuela es solo para unos...


#### Medida de información mutua

In [11]:
text_elfinanciero = " ".join(df_elfinanciero_titles["title_clean"].tolist())

tokens_elfinanciero = tokenize_words(text_elfinanciero)
print(f"Número de tokens: {len(tokens_elfinanciero)}")  

bigrams_str_financiero = ngrams_fn(tokens_elfinanciero, n=2)

bigrams_elfinanciero = bigram_counter_from_str_keys(bigrams_str_financiero)
print(f"Número de bigramas: {len(bigrams_elfinanciero)}")

pmi_elfinanciero = pmi_df(tokens_elfinanciero, bigrams_elfinanciero, min_freq=MIN_FREQ)

# Mostramos los top-k
top_pmi(pmi_elfinanciero, k=TOP_K)

Número de tokens: 325811
Número de bigramas: 142681


,w1,w2,freq,pmi
0,rutilio,escandon,10,14.991752
1,rosita,pelayo,10,14.991752
2,britney,spears,10,14.854249
3,salomon,jara,10,14.716745
4,galilea,montijo,13,14.613240
5,cristiano,ronaldo,12,14.613240
6,hipolito,mora,11,14.613240
7,natanael,cano,10,14.613240
8,octavio,ocana,10,14.591214
9,tenoch,huerta,12,14.506325
